# 1. 라이브러리 선언

In [2]:
pwd

'C:\\gitclone\\python\\lecture\\python-lecture\\Session14 - LLM모델 연동 실습'

In [1]:
from langchain_community.document_loaders import PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings

import re
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

from sentence_transformers import SentenceTransformer
from datasets import load_dataset

import os

from langchain.chains.question_answering import load_qa_chain

from langchain.chains import RetrievalQA
from langchain.memory import ConversationSummaryMemory
from langchain.text_splitter import CharacterTextSplitter
import chromadb

from langchain.prompts import PromptTemplate
from langchain.llms import Ollama

# 2. 벡터DB 내 PDF 정보저장

In [2]:
ST = SentenceTransformer("jhgan/ko-sbert-sts")

# %cd /content/
loader = PyPDFLoader('./pdfdata//강서캠퍼스.pdf')
documents = loader.load()
output = []
# text 정제
for page in documents:
    text = page.page_content
    output.append(text)

doc_chunks = []
for line in output:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300, # 최대 청크 길이
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""], #  텍스트를 청크로 분할하는 데 사용되는 문자 목록
        chunk_overlap=200, # 인접한 청크 간에 중복되는 문자 수
    )
    chunks = text_splitter.split_text(line)
    for i, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk, metadata={ "source": '강서캠퍼스.pdf', "page": i}
        )
        doc_chunks.append(doc)

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
embedding_func = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-sts"
)

vectordb  = Chroma.from_documents(doc_chunks, embedding_func)

In [8]:
# ollamaModel = "hkcode-beomi-8b-ko-llama3-model"
# ollamaModel = "llama3-hkcode-ko"
# ollamaModel = "llama3-hkcode-unsloth"
ollamaModel = 'llama3-hkcode-meta' #You can replace the model name if needed

# template = """<s>[INST] <<SYS>>please answer in korean language </SYS>>Given the context - {context} </s>[/INST] [INST] Answer the following - {question}[/INST]"""

template="""you are a assistant please answer in korean lanauage {context}

### Instruction:
{question}

### Response:"""
pt = PromptTemplate(
            template=template, input_variables=["context", "question"]
        )

rag = RetrievalQA.from_chain_type(
            llm=Ollama(model=ollamaModel),
            retriever=vectordb.as_retriever(),
            # memory=ConversationSummaryMemory(llm = Ollama(model=ollamaModel)),
              return_source_documents=True,
            chain_type_kwargs={"prompt": pt, "verbose": True},
        )
rag.invoke("스마트금융과 전화번호는 어떻게 되나요?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
you are a assistant please answer in korean lanauage - 8 -9 등록포기 안내  구분 내용 등록포기 방법▪ 온라인 (홈페이지 )으로 본인이 직접 등록포기 ▪ 등록포기 신청이 승인처리 되면 취소할 수 없으니 유의하기 바람 10 학과 및 기타   <하이테크과정 학과> 학과 내용 스마트금융 ☎02-2186-5861▪ 금융과 소프트웨어 기술을 융합한 핀테크 산업 분야의 실무형 개발자 양성 ▪ (주요교과목 ) 빅데이터 분석, 블록체인 개발, 금융데이터 분석, 웹 어플리케이션 개발 ▪ (취득가능자격증 ) 정보처리기사 , 정보처리산업기사 , 리눅스마스터 , SQLD, OCJP

- 1 -한국폴리텍대학 서울강서캠퍼스 2024 학년도 하이테크과정 모집요강 (주소) 서울특별시 강서구 우장산로 10길 112 (대표번호 ) 02-2186-5800 1 모집학과 (직종) 및 모집인원 가.선발 비율 구분 정원 모집 1차 모집 2차 하이테크 100% 50% 50% ※ 정원의 110%까지 선발 ※ 모집 2차 실시 후 결원이 발생한 경우는 자율 모집 ※ 모집 차수별 미달 인원은 이월하여 다음 차수에 모집(모집1차→모집2차→자율모집 ) ※ 모집 1차 합격 후 등록자는 모집 2차 및 자율 모집에 지원할 수 없음 나

, 개별 공지하지 않음) ■ 지원자별 해당 제출서류 주소지 (등기우편 )  - (07684) 서울특별시 강서구 우장산로 10길 112    한국폴리텍대학 서울강서캠퍼스 교학처    (하이테크과정 입시담당자 앞)  ※ 입학전형은 대학 사정에 따라 입학전형관리소위원회의 심의를 통하여 변경될 수 있음과 정 학 과 직 종모집 정원모집 인원모집 1차 모집 2차 소계 일반 우선 소계 일반 우선 하이테크 (주간1년)총 계 100 108 56 52 4 52 48 4 사

{'query': '한국폴리텍대학 하이테크과정 스마트금융과 모집정원은 어떻게 되나요?',
 'result': '한국폴리텍대학 하이테크 과정 스마트금융과는 50명 입니다.',
 'source_documents': [Document(page_content='- 8 -9 등록포기 안내  구분 내용 등록포기 방법▪ 온라인 (홈페이지 )으로 본인이 직접 등록포기 ▪ 등록포기 신청이 승인처리 되면 취소할 수 없으니 유의하기 바람 10 학과 및 기타   <하이테크과정 학과> 학과 내용 스마트금융 ☎02-2186-5861▪ 금융과 소프트웨어 기술을 융합한 핀테크 산업 분야의 실무형 개발자 양성 ▪ (주요교과목 ) 빅데이터 분석, 블록체인 개발, 금융데이터 분석, 웹 어플리케이션 개발 ▪ (취득가능자격증 ) 정보처리기사 , 정보처리산업기사 , 리눅스마스터 , SQLD, OCJP', metadata={'page': 0, 'source': '강서캠퍼스.pdf'}),
  Document(page_content='- 1 -한국폴리텍대학 서울강서캠퍼스 2024 학년도 하이테크과정 모집요강 (주소) 서울특별시 강서구 우장산로 10길 112 (대표번호 ) 02-2186-5800 1 모집학과 (직종) 및 모집인원 가.선발 비율 구분 정원 모집 1차 모집 2차 하이테크 100% 50% 50% ※ 정원의 110%까지 선발 ※ 모집 2차 실시 후 결원이 발생한 경우는 자율 모집 ※ 모집 차수별 미달 인원은 이월하여 다음 차수에 모집(모집1차→모집2차→자율모집 ) ※ 모집 1차 합격 후 등록자는 모집 2차 및 자율 모집에 지원할 수 없음 나', metadata={'page': 0, 'source': '강서캠퍼스.pdf'}),
  Document(page_content=', 개별 공지하지 않음) ■ 지원자별 해당 제출서류 주소지 (등기우편 )  - (07684) 서울특별시 강서구 우장산로 10길 112    한국폴리텍대학 서울강서캠퍼스 교학처    (하이테크과정 입시담당자 앞)  ※ 입학전형은 대학

In [7]:
resp = rag.invoke("스마트금융과 전화번호는 어떻게 되나요?")
resp["result"]



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
you are a assistant please answer in korean lanauage - 8 -9 등록포기 안내  구분 내용 등록포기 방법▪ 온라인 (홈페이지 )으로 본인이 직접 등록포기 ▪ 등록포기 신청이 승인처리 되면 취소할 수 없으니 유의하기 바람 10 학과 및 기타   <하이테크과정 학과> 학과 내용 스마트금융 ☎02-2186-5861▪ 금융과 소프트웨어 기술을 융합한 핀테크 산업 분야의 실무형 개발자 양성 ▪ (주요교과목 ) 빅데이터 분석, 블록체인 개발, 금융데이터 분석, 웹 어플리케이션 개발 ▪ (취득가능자격증 ) 정보처리기사 , 정보처리산업기사 , 리눅스마스터 , SQLD, OCJP

- 1 -한국폴리텍대학 서울강서캠퍼스 2024 학년도 하이테크과정 모집요강 (주소) 서울특별시 강서구 우장산로 10길 112 (대표번호 ) 02-2186-5800 1 모집학과 (직종) 및 모집인원 가.선발 비율 구분 정원 모집 1차 모집 2차 하이테크 100% 50% 50% ※ 정원의 110%까지 선발 ※ 모집 2차 실시 후 결원이 발생한 경우는 자율 모집 ※ 모집 차수별 미달 인원은 이월하여 다음 차수에 모집(모집1차→모집2차→자율모집 ) ※ 모집 1차 합격 후 등록자는 모집 2차 및 자율 모집에 지원할 수 없음 나

, 개별 공지하지 않음) ■ 지원자별 해당 제출서류 주소지 (등기우편 )  - (07684) 서울특별시 강서구 우장산로 10길 112    한국폴리텍대학 서울강서캠퍼스 교학처    (하이테크과정 입시담당자 앞)  ※ 입학전형은 대학 사정에 따라 입학전형관리소위원회의 심의를 통하여 변경될 수 있음과 정 학 과 직 종모집 정원모집 인원모집 1차 모집 2차 소계 일반 우선 소계 일반 우선 하이테크 (주간1년)총 계 100 108 56 52 4 52 48 4 사

'한국폴리텍대학의 홈페이지에 들어가면 전체 정원이 나옵니다.'

# gradio 챗봇

In [10]:
# gradio == 3.45.0

In [ ]:
import requests, json

import gradio as gr

model = 'llama3-hkcode-meta' #You can replace the model name if needed
# model = 'llama3' #You can replace the model name if needed
context = [] 



import gradio as gr

#Call Ollama API
def generate(prompt, context, top_k, top_p, temp):
    resp = rag.invoke(prompt)
    # r = resp["result"]

    response = resp["result"]  


    context = []
    return response, context



def chat(input, chat_history, top_k, top_p, temp):

    chat_history = chat_history or []

    global context
    output, context = generate(input, context, top_k, top_p, temp)

    chat_history.append((input, output))

    return chat_history, chat_history
  #the first history in return history, history is meant to update the 
  #chatbot widget, and the second history is meant to update the state 
  #(which is used to maintain conversation history across interactions)


#########################Gradio Code##########################
block = gr.Blocks()


with block:

    gr.Markdown("""<h1><center> Jarvis </center></h1>
    """)

    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder="Type here")

    state = gr.State()
    with gr.Row():
        top_k = gr.Slider(0.0,100.0, label="top_k", value=40, info="Reduces the probability of generating nonsense. A higher value (e.g. 100) will give more diverse answers, while a lower value (e.g. 10) will be more conservative. (Default: 40)")
        top_p = gr.Slider(0.0,1.0, label="top_p", value=0.9, info=" Works together with top-k. A higher value (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. (Default: 0.9)")
        temp = gr.Slider(0.0,2.0, label="temperature", value=0.8, info="The temperature of the model. Increasing the temperature will make the model answer more creatively. (Default: 0.8)")


    submit = gr.Button("SEND")

    submit.click(chat, inputs=[message, state, top_k, top_p, temp], outputs=[chatbot, state])


block.launch(debug=True,share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://8a0482244cfbc66f05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
you are a assistant please answer in korean lanauage 훈련개시일 전일까지 고용보험 상실 처리가 되지 않은 자   - 폴리텍대학 내 과정 구분 없이 이중 등록이 된 자   - 기타 부정한 방법으로 합격 또는 입학한 사실이 확인된 자 기 타▪ 모집 요강에 구체적으로 명시되지 않은 사항은 대학의 입학전형관리 (소)위원회에서 결정 ▪ 입학전형은 대학의 사정에 따라 변경될 수 있음 문의사항  * 서 울 강 서 캠 퍼 스  : 02-2186-5800    (스 마 트금 융 과 ) 02-2186-5861 (사이버보안과 ) 02-2186-5850 (출판편집디자인과 ) 02-2186-5920 구분

,000원(월 20만원 한도)       ☞ 그외 훈련생 : 출석 1일 3,300원(월 66,000 원 한도)     - 교통비 : 출석 1일 2,500원(월 5만원 한도)    ※※ 서울강서캠퍼스는 기숙사 없음※※  <오시는 길>      대중교통 교통편 노   선 오시는 길 지하철 5호선 (우장산역 )  9호선 (가양역 )2번 출구 도보 10분 8번 출구 버스 10분 버스한국폴리텍 서 울 강 서 대 학 교604,606, 650,  652, 673,  5712, 6514,  6627, 6629,  70, 70-2

- 8 -9 등록포기 안내  구분 내용 등록포기 방법▪ 온라인 (홈페이지 )으로 본인이 직접 등록포기 ▪ 등록포기 신청이 승인처리 되면 취소할 수 없으니 유의하기 바람 10 학과 및 기타   <하이테크과정 학과> 학과 내용 스마트금융 ☎02-2186-5861▪ 금융과 소프트웨어 기술을 융합한 핀테크 산업 분야의 실무형 개발자 양성 ▪ (주요교과목 ) 빅데이터 분석, 블록체인 개발, 금융데이터 분석, 웹 어플리케이션 개발 ▪ (취득가능자격증 )

In [ ]:
import requests, json

import gradio as gr

model = 'llama3-hkcode-meta' #You can replace the model name if needed
# model = 'llama3' #You can replace the model name if needed
context = [] 



import gradio as gr

#Call Ollama API
def generate(prompt, context, top_k, top_p, temp):
    r = requests.post('http://localhost:11434/api/generate',
                     json={
                         'model': model,
                         'prompt': prompt,
                         'context': context,
                         'options':{
                             'top_k': top_k,
                             'temperature':top_p,
                             'top_p': temp
                         }
                     },
                     stream=False)
    r.raise_for_status()

 
    response = ""  

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        print(response_part)
        if 'error' in body:
            raise Exception(body['error'])

        response += response_part

        if body.get('done', False):
            context = body.get('context', [])
            return response, context



def chat(input, chat_history, top_k, top_p, temp):

    chat_history = chat_history or []

    global context
    output, context = generate(input, context, top_k, top_p, temp)

    chat_history.append((input, output))

    return chat_history, chat_history
  #the first history in return history, history is meant to update the 
  #chatbot widget, and the second history is meant to update the state 
  #(which is used to maintain conversation history across interactions)


#########################Gradio Code##########################
block = gr.Blocks()


with block:

    gr.Markdown("""<h1><center> Jarvis </center></h1>
    """)

    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder="Type here")

    state = gr.State()
    with gr.Row():
        top_k = gr.Slider(0.0,100.0, label="top_k", value=40, info="Reduces the probability of generating nonsense. A higher value (e.g. 100) will give more diverse answers, while a lower value (e.g. 10) will be more conservative. (Default: 40)")
        top_p = gr.Slider(0.0,1.0, label="top_p", value=0.9, info=" Works together with top-k. A higher value (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. (Default: 0.9)")
        temp = gr.Slider(0.0,2.0, label="temperature", value=0.8, info="The temperature of the model. Increasing the temperature will make the model answer more creatively. (Default: 0.8)")


    submit = gr.Button("SEND")

    submit.click(chat, inputs=[message, state, top_k, top_p, temp], outputs=[chatbot, state])


block.launch(debug=True,share=True)